# విషయ సూచిక
- [మార్గదర్శకత్వం గురించి](../../../../code/01.Introduce)
- [సెట్టప్](../../../../code/01.Introduce)
- [అనియంత్రిత ఉత్పత్తి](../../../../code/01.Introduce)
- [Phi 3 తరఫున మాట్లాడటం](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [ఎంపిక](../../../../code/01.Introduce)
- [ఆలోచనా గొలుసు](../../../../code/01.Introduce)
- [JSON ఉత్పత్తి](../../../../code/01.Introduce)
- [HTML ఉత్పత్తి](../../../../code/01.Introduce)


# Guidance గురించి
Guidance అనేది ఏదైనా భాషా మోడల్ (LM) యొక్క అవుట్‌పుట్‌లను నియంత్రించడానికి నిరూపితమైన ఓపెన్-సోర్స్ Python లైబ్రరీ. ఒకే API కాల్‌తో, మీరు (Python లో) మోడల్ అనుసరించవలసిన ఖచ్చితమైన ప్రోగ్రామాటిక్ పరిమితులను వ్యక్తపరచి, JSON, Python, HTML, SQL లేదా వినియోగ సందర్భాన్ని సంపూర్ణంగా తీర్చే ఏదైనా నిర్మిత రూపంలో అవుట్పుట్ ఉత్పత్తి చేయవచ్చు.

Guidance మామూలు ప్రాంప్టింగ్ పద్ధతుల నుండి భిన్నంగా ఉంటుంది.  ఇది ఇన్ఫరెన్స్ లేయర్‌లో మోడల్‌ను టోకెన్ వారీగా నడిపి పరిమితులను అమలు చేస్తుంది, ఫలితంగా ఉన్నతమైన నాణ్యతా అవుట్‌పుట్‌లు వస్తాయి మరియు అత్యంత నిర్మాణాత్మక పరిస్థితుల్లో ఉపయోగించబడినప్పుడు ఖర్చు మరియు ఆలస్యం ను 30–50% వరకు తగ్గిస్తుంది.

Guidance గురించి మరింత తెలుసుకోవడానికి, [GitHub లోని సార్వజనిక రిపోజిటరీ](https://github.com/guidance-ai/guidance) ని సందర్శించండి లేదా Microsoft Build లోని [Guidance బ్రేక్‌అవుట్ సెషన్](https://www.youtube.com/watch?v=qXMNPVVlCMs) ను వీక్షించండి.


# సెటప్
1. Guidance ను `pip install guidance --pre` ద్వారా ఇన్‌స్టాల్ చేయండి
2. Azureలో Phi 3.5 mini ఎండ్‌పాయింట్‌ను డిప్లాయ్ చేయడానికి https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml కి వెళ్లి "Deploy" బటన్‌పై క్లిక్ చేయండి
3. మీ ఎండ్‌పాయింట్ యొక్క API కీని `AZURE_PHI3_KEY` అనే ఎన్వైరాన్‌మెంట్ వేరియబుల్‌లో మరియు URLని `AZURE_PHI3_URL` అనే ఎన్వైరాన్‌మెంట్ వేరియబుల్‌లో నిల్వ చేయండి


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# పరిమితులేని ఉత్పత్తి
టెక్స్ట్‌ను `gen()` ఫంక్షన్ ఉపయోగించి ఎటువంటి పరిమితులకుండానే ఉత్పత్తి చేయవచ్చు. ఇది Guidance లేకుండా మోడల్‌ను ఉపయోగించడం సమానమే.

## చాట్ ఫార్మాటింగ్
బహుళ చాట్ మోడళ్లలా, Phi-3 వినియోగదారు మరియు సహాయకుడు మధ్య సందేశాలను ఒక నిర్దిష్ట ఫార్మాట్‌లో ఆశిస్తుంది. Guidance, Phi-3 యొక్క చాట్ టెంప్లేట్‌కు మద్దతు ఇస్తుంది మరియు మీ కోసం చాట్ ఫార్మాటింగ్‌ను నిర్వహిస్తుంది. చాట్ టర్న్‌లను సృష్టించడానికి, సంభాషణలోని ప్రతి భాగాన్ని `with user()` లేదా `with assistant()` బ్లాక్‌లో ఉంచండి. `with system()` బ్లాక్‌ను సిస్టమ్ సందేశాన్ని సెట్ చేయడానికి ఉపయోగించవచ్చు.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## టోకెన్ల ఆదా
In highly structured scenarios, Guidance can skip tokens and generate only necessary tokens, improving performance, increasing efficiency and saving API costs. Generated tokens are shown in this notebook with a highlighted background. Forced tokens are shown without highlighting and cost the same as input tokens, which are estimated at one third the cost of output tokens.

*గమనిక:* పరిమితులలేని జనరేషన్ ఉన్న మొదటి ఉదాహరణలో మేము ఎలాంటి పరిమితులును అందించడం లేదందున ఏ టోకెన్లను బలవంతంగా ఉత్పత్తి చేయడం సాధ్యమవలేదు.


# Phi 3 కోసం మాట్లాడటం 
Guidance ఉపయోగించి, మీరు సులభంగా టెక్స్ట్‌ను మోడల్ యొక్క ప్రతిస్పందనల్లో చేర్చవచ్చు. ఇది మీరు మోడల్ యొక్క అవుట్‌పుట్‌ను నిర్దిష్ట దిశలో మార్గనిర్దేశం చేయాలనుకుంటే సహాయపడుతుంది.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

`ఆస్ట్రేలియాలో రాజధాని ఉంది` హైలైట్ చేయబడలేదు ఎందుకంటే అసిస్టెంట్ యొక్క ప్రతిస్పందనలో ఆ భాగం Guidance ద్వారా బలవంతంగా అమలు చేయబడింది.


# regex తో పరిమితం చేయడం
మునుపటి ఉదాహరణలో, Phi 3 `Canberra` అని ప్రశ్నకు సమాధానం చెప్పిన తర్వాత అనుసరణ వివరణలుతో స్పందించింది. మోడల్ యొక్క అవుట్పుట్‌ను ఖచ్చితంగా ఒకే పదంగా పరిమితం చేయడానికి, regex ఉపయోగించవచ్చు.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

రెగ్యులర్ ఎక్స్‌ప్రెషన్‌తో, కేవలం `Canberra` అనే పదం మాత్రమే ఉత్పత్తి అవుతుంది.


# బహుళ ఎంపికల నుండి ఎంచుకోవడం
కొన్ని సాధ్యమైన ఎంపికలు తెలిసినప్పుడు, ఎంపికల జాబితా నుండి ఒకటి ఎంచుకోవడానికి మీరు `select()` ఫంక్షన్‌ను ఉపయోగించవచ్చు.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

`select()`తో, కేవలం `Can` అనే టోకెన్ మాత్రమే ఉత్పత్తి అయ్యింది. `Canberra` మాత్రమే స్పందనను పూర్తిచేయగల సాధ్యమైన ఏకైక ఎంపిక కావడంతో, మిగతా టోకెన్‌లు బలవంతంగా నిర్ణయించబడ్డాయి.


# ఆలోచనా శృంఖల

ఆలోచనా శృంఖల ఒక సాంకేతిక పద్ధతి ఇది, సమస్యను దశలవారీగా ప్రాసెస్ చేయమని ప్రేరేపించడం ద్వారా మోడల్ యొక్క అవుట్పుట్ నాణ్యతను మెరుగుపరచడంలో సహాయపడుతుంది. సాధారణంగా, తుది సమాధానానికి చేరుకోవడానికి బహుళ ప్రాంప్ట్ రౌండ్లు అవసరం. మొదట, మోడల్‌ను దశలవారిగా ఆలోచించమని సూచించండి. తర్వాత, మళ్లీ మోడల్‌ను ప్రాంప్ట్ చేసి తుది సమాధానం ఇవ్వమని కోరండి. సాధారణ చాట్ ఇన్ఫరెన్స్ APIలతో, ఇది 2 API calls తీసుకుంటుంది, మరియు మోడల్ రూపొందించిన “ఆలోచనా శృంఖల” రెండుసార్లు చార్జ్ చేయబడుతుంది – ఒకసారి మోడల్ దాన్ని రూపొందించినప్పుడు అవుట్‌పుట్ టోకెన్లుగా, మరియు రెండవ కాల్ కోసం ఇన్‌పుట్ టోకెన్లుగా. Guidanceతో,  మొత్తం బహుళ-దశ ప్రక్రియను ఒకే API కాల్ భాగంగా ప్రాసెస్ చేసి చార్జ్ చేయబడుతుంది, ఖర్చు మరియు లేటెన్సీ తగ్గుతాయి.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON ఉత్పత్తి
Guidance ను ఉపయోగించి JSON schema లేదా pydantic మోడల్‌కు అనుగుణంగా ఉండే JSON రూపొందించబడుతుందని హామీ చేయవచ్చు, ఉదాహరణకు ఇక్కడ చూపించిన యూజర్ ప్రొఫైల్ స్కీమా.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML సృష్టి

Guidance ను కోడ్ జనరేట్ చేయడానికి మరియు ప్రోగ్రామింగ్ భాషలోని వ్యాకరణ అవసరాలను అనుసరించడానికి కూడా ఉపయోగించవచ్చు. ఈ విభాగంలో, చాలా సాదా HTML వెబ్‌పేజీలు రాయడానికి ఒక చిన్న Guidance ప్రోగ్రామ్‌ను తయారుచేయబోతున్నాం.

We will break the webpage down into smaller sections, each with its own Guidance function.  ఆవి తరువాత మా తుది ఫంక్షన్‌లో కలిపి ఒక HTML వెబ్‌పేజీని తయారు చేస్తాము.
We will then run this function against a Guidance-enabled model in Azure AI.

*గమనిక:* ఇది ఒక పూర్తి లక్షణాలున్న HTML జనరేటర్ కాదు; లక్ష్యం మీ వ్యక్తిగత అవసరాల కోసం మీరు ఎలా నిర్మిత అవుట్పుట్ సృష్టించవచ్చో చూపించడం

Guidance నుండి మాకు అవసరమైన వాటిని దిగుమతి చేయడంతో ప్రారంభిస్తాము:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML వెబ్‌పేజీలు చాలా నిర్మితమైనవి, మరియు మనం Guidance ఉపయోగించి పేజీలోని ఆ భాగాలను 'బలవంతం' చేస్తాము.
మనం స్పష్టంగా మోడల్ నుంచి టెక్స్ట్‌ను కోరినప్పుడు, అది ట్యాగ్ కావొచ్చని భావించబడే ఏదైనా కలిగి ఉండకూడదని నిర్ధారించుకోవాలి - అంటే మనం '<' మరియు '>' అక్షరాలను తప్పించాలి:


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

ఆపై ఈ ఫంక్షన్‌ను ఉపయోగించి ఏదైనా HTML ట్యాగ్‌లో టెక్స్ట్‌ను రూపొందించవచ్చు:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

ఇప్పుడు, మనం పేజీ హెడ్డర్‌ను సృష్టిద్దాం.
దీనిలో భాగంగా, మనం ఒక పేజీ శీర్షికను తయారుచేయాలి:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML పేజీ యొక్క బాడీ శీర్షికలు మరియు పేరాగ్రాఫ్‌లతో నింపబడుతుంది.
ప్రతి ఒక్కదానికి ఒక ఫంక్షన్‌ను నిర్వచించవచ్చు:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

ఇప్పుడు, HTML యొక్క శరీరాన్ని నిర్వచించే ఫంక్షన్.
ఇది `select()` ను `recurse=True` తో ఉపయోగించి బహుళ శీర్షికలు మరియు పేరాగ్రాఫ్‌లను సృష్టించడానికి:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

తర్వాత, పూర్తి HTML పేజీని రూపొందించే ఫంక్షన్‌కు మనం వస్తాము.
మేము HTML ప్రారంభ ట్యాగ్‌ను జతచేసి, ఆ తర్వాత హెడర్‌ను రూపొందించి, తర్వాత బాడీని, మరియు చివరగా ముగింపు HTML టాగ్‌ను జతచేస్తాము:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

మేము వినియోగదారులకు అనుకూలమైన ఒక రాపర్‌ను అందిస్తున్నాము, ఇది మనకు వీటిని చేయడానికి అనుమతిస్తుంది:
- జెనరేషన్ ఉష్ణోగ్రతను సెట్ చేయడం
- Model object నుండి ఉత్పత్తి చేయబడిన పేజీని క్యాప్చర్ చేయడం


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

మనం మోడల్‌కు ఒక ప్రాంప్ట్ అందించి, ఆపై ఒక జనరేషన్‌ను అభ్యర్థించవచ్చు:


In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

అప్పుడు మనం అవుట్పుట్‌ను ఒక ఫైల్‌లో రాయవచ్చు:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

మరియు [ఫలితాన్ని చూడండి](../../../../code/01.Introduce/sample_page.html).


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
నిరాకరణ:
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించారు. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, స్వయంచాలక అనువాదాల్లో తప్పులు లేదా లోపాలు ఉండే అవకాశం ఉందని దయచేసి గమనించండి. మూల పత్రాన్ని దాని స్థానిక (మూల) భాషలోని ప్రతిని అధికారిక మూలంగా పరిగణించండి. కీలక సమాచారానికి వృత్తిపరమైన మానవ అనువాదం చేయించుకోవడమే సిఫార్సు చేయబడుతుంది. ఈ అనువాదం ఉపయోగించడంతో కలిగే ఏవైనా అపార్థాలు లేదా తప్పుగా అర్థం చేసుకోవడాలపై మేము బాధ్యత వహించమని చెప్పలేము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
